<a href="https://colab.research.google.com/github/metamatematico/Experimentos-IA/blob/master/Creador_de_poes%C3%ADa%2C_ajustado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferir el estilo de una obra textual a un modelo de escritura automatizada

Para leer el contexto en que se generó la siguiente receta leer sobre el  [Taller de escritura computarizada](https://www.notion.so/ivanvladimir/Taller-de-escritura-computarizada-e7a7bc49b552475a92b45db6416437cd)

El proceso de adaptación se lleva a cabo con los siguiente pasos
1. Instalar las librerías de python que permiten la generación
2. Conectarnos al google drive para guardar nuestro modelo y/o acceder la obra textual
3. Obtener obra textual de la que se obtendrá el estilo
4. Entrenar
5. Generar

Si ya se cuenta con un modelo entrenado, es posible brincarse el paso de entrenaiento y obtener obra (3 y 4) y pasar directamente a generar directamente

## 1. Se instalan las librerias adecuadas

Las siguentes librerias hechas por la empresa [Hugginface](https://huggingface.co/) permiten la adaptación (entrenamiento) y generación automatizada de textos. Las librerías son:
1. _transformers_, librería general para manejar modelos basados en la arquitectura neruonal transformers
2. _datasets_, librería asociada a transoformers para la manipulación de dataset/corpus de datos

Al ejecutar la celda, esperar mensajes de que se descacargan archivos y se instala módulos de python.

In [2]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.1MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 47.9MB/s 
     |████████████████████████████████| 901kB 58.1MB/s 
     |████████████████████████████████| 225kB 10.1MB/s 
     |████████████████████████████████| 245kB 13.6MB/s 
     |████████████████████████████████| 112kB 17.1MB/s 


In [3]:
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)


## Obtener script de adaptaición (cont. 1)

La transferencia de estilo, adaptación la llevaremos a cabo usando un script que forma parte de la librería de 'transformers' para lo cual es necesario descargarlo con la siguiente celda.

Esperar unmensaje de que se descarga el archivo 'run_clm.py'

In [4]:
!wget -O run_clm.py https://raw.githubusercontent.com/huggingface/transformers/v4.4.1/examples/language-modeling/run_clm.py

--2021-04-26 18:03:59--  https://raw.githubusercontent.com/huggingface/transformers/v4.4.1/examples/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18700 (18K) [text/plain]
Saving to: ‘run_clm.py’

run_clm.py          100%[===================>]  18.26K  --.-KB/s    in 0.001s  

2021-04-26 18:03:59 (28.9 MB/s) - ‘run_clm.py’ saved [18700/18700]



## 2. Conectar nuestro google drive

La plataforma colab puede acceder a nuestro google drive, en particular esto nos sirve para guardar el modelo adaptado en google drive y volverlo a usarlo más adelante (recordar que la máquina virtual en la que se ejecua colab deja de existir depués de un tiempo de inactividad o máximo 8 horas). 

Para lograr la conección, ejecutar la celda, y hacer click en el link generado automáticamente; confirmar con el sistema google la conexión hasta identificar el código de acceso que se pegará en la caja de texto de la misma celda. Depupués de unos segundos aparecerá el mensaje que google drive se ha 'montado' en ua ruta "/content/gdrive"

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


## 3. Conseguir el texto que se utilizará para adaptar

Al proceso de adaptación hay que pasarla una obra que se adaptará, para lograr esto es necesario hacerla disponible dese la plataforma de colab, se pude hacer de tres forma:

1. Bajarlo desde el internet directamente al ambiente de Colab
2. Subirlo a colab a través del menu "Files" ubicado en la parte izquierda de la plataform
3. Usando la conexión con google drive

En general se recomienda el formato .txt en una codificiación UTF-8; también es importante tener muy claro el nombre del archivo que contiene el texto y la ruta en la que se encuentra. Si se hace a través de los pasos 3.1 y 3.2; lo más probable es que esté accesibe desde el directotio de ejecución de colab, por lo que no haría falta fijarse en la ruta.


In [6]:
import os
import re

contenido= list()

def callback(pat):
  return "\n\n"+pat.group(1).upper()+"\n\n"

def callback2(pat):
  return pat.group(1).upper()+"\n\np* --"

with open('/content/gdrive/MyDrive/Proyecto NLP/archivos de ejecucion/full_poemas_rosa_espino_manipulado.txt') as archivo:
       for linea in archivo:
            if not linea == '\n':
                linea.rstrip('\n')
                linea2 = ' -- '+linea+' -- \n'
            else:
                linea2 = '\n'
            
            l = linea2.split('\n --')
            
            if l == ['\n']:
                l = '*p'.join(l)
                #print(p)
                contenido.append('*p \n'+ l + '*p')
                
            elif not l == ['\n'] :
                l = ' -- '.join(l)
                contenido.append((l))

       contenido="".join(contenido)
       contenido=re.sub(r"\n\*p \n\n","\n\n",contenido,re.DOTALL)
       contenido=re.sub(r"\n\n(.*?)\n\n",callback,contenido,re.DOTALL)
       contenido=re.sub(r"(\n\*p --[A-ZÑy .]*?--\s+?)\n --",callback2,contenido,re.DOTALL)

with open('/content/gdrive/MyDrive/Proyecto NLP/archivos de ejecucion/full_poemas_rosa_espino_manipulado.txt', 'w',  encoding='utf-8', errors='ignore') as archivo:
    archivo.writelines(contenido)

### 3.1 Bajarlo desde internet

Usando el comando wget obtener el archivo de una URL pública con el texto. En la celda de abajo remplazar por la URL para bajar el texto recomendado. En el ejemplo se descarga El quijote en un archivo "el_quijote.txt".

Quidado, si la celda se ejecuta varias veces con el mismo nombre de archivo a bajar se crearán varias versiones del texto con números indicando la versión.

Se espera que el archivo recuperado se baje y quede visible en el directorio de ejecución de colab

In [ ]:
#!wget https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt

--2021-03-12 00:44:07--  https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1060259 (1.0M) [text/plain]
Saving to: ‘el_quijote.txt’

el_quijote.txt      100%[===================>]   1.01M  --.-KB/s    in 0.02s   

2021-03-12 00:44:08 (51.4 MB/s) - ‘el_quijote.txt’ saved [1060259/1060259]



### 3.2 Subirlo a colab

Si el archivo está de forma local en la máquina utilizada para acceder a la plataforma colab, es posible subirlo a ésta. Para este proceso se usa el menu 'Files' de las izquierda siguiendo el procedimiento después de hacer click en el botón de 'Upload'. No es necesario ejecutar ninguna celda.

Se espera que el archivo subido quede visible en el directorio de ejecución de colab

### 3.3 A través de google drive

Si el archivo está en el servicio de nube de google drive, es posible acceder a ese archivo ya que tenemos conectado nuestro google drive a colab (ver paso 2). Para identificar la ruta y nombre del archivo, se puede usar el menú "Files" a la izquieda de la plataforma colab, localizar el archivo en el google drive, bajo el nombre oprimir el menu con tres punto "..." y escoger la opción "Copiar path". No es necesario ejecutar ninguna celda.

Se espera haber identificado la ruta del archivo dentro de google drive que se usará como fuente de estilo.

## 4. Entrenamiento para adpatar el modelo

La siguiente celda es la que adapta un modelo pre entrenado 'datificate/gpt2-small-spanish' con el estilo del texto fuente.

En este paso será necesario tener claro el nombre y la ruta (para paso 3.3) del archivo textual que se usará como fuente para el estilo (ver paso 3). Esa ruta (de ser necesaria) y nombre se sustituira en la celda de abajo para la opción '--train_file' 

Cuidado en el formato de la celda es muy imporante mantener el símbolo '\' al final de la línea en la opción '--train_file' y que no haya ningún símbolo o espacio después de éste.

Al ejecutar la siguiente línea puede llevarse varios minutos o hasta horas

In [7]:
!mkdir output
!python run_clm.py \
    --model_name_or_path 'datificate/gpt2-small-spanish' \
    --train_file '/content/gdrive/MyDrive/Proyecto NLP/archivos de ejecucion/full_poemas_rosa_espino_manipulado.txt' \
    --do_train \
    --block_size 128\
    --num_train_epochs 30 \
    --overwrite_output_dir \
    --save_total_limit 0 \
    --gradient_accumulation_steps=12 \
    --learning_rate=3e-5 \
    --seed 10 \
    --fp16 \
    --fp16 --fp16_opt_level O3 \
    --prediction_loss_only \
    --per_device_train_batch_size 4 \
    --validation_file "/content/gdrive/MyDrive/Proyecto NLP/archivos de ejecucion/full_poemas_rosa_espino_manipulado.txt" \
    --do_eval \
    --output_dir /output/gpt2-small-spanish-adaptado

mkdir: cannot create directory ‘output’: File exists
2021-04-26 18:04:51.591941: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/26/2021 18:04:53 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
04/26/2021 18:04:53 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/output/gpt2-small-spanish-adaptado, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=True, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=12, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=30.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Apr26_18-04-52_2ac

## 4.2 Copiar modelo a google drive

La siguiente celda copia el modelo a una carpta dentro de google drive llamada "models" si se quiere poner en otra carpeta adaptar la celda.

In [8]:
# Moviendo el modelo al google drive
output_dir = '/content/gdrive/My Drive/models/'
!mkdir -p '/content/gdrive/My Drive/models/'
!cp -r '/output/gpt2-small-spanish-adaptado' '{output_dir}'
!ls '{output_dir}/gpt2-small-spanish-adaptado'

all_results.json  config.json		   tokenizer_config.json
checkpoint-1000   eval_results.json	   trainer_state.json
checkpoint-1500   merges.txt		   training_args.bin
checkpoint-2000   pytorch_model.bin	   train_results.json
checkpoint-500	  special_tokens_map.json  vocab.json


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 4.3 Más opciones al script

La siguiente celda muestra todas las opciones diponibles en el texto, por si se requiere hacer de modificaciones al entrenamiento

In [ ]:
!python run_clm.py -h

2021-02-22 23:27:55.466270: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
usage: run_clm.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                  [--model_type MODEL_TYPE] [--config_name CONFIG_NAME]
                  [--tokenizer_name TOKENIZER_NAME] [--cache_dir CACHE_DIR]
                  [--no_use_fast_tokenizer]
                  [--use_fast_tokenizer [USE_FAST_TOKENIZER]]
                  [--model_revision MODEL_REVISION]
                  [--use_auth_token [USE_AUTH_TOKEN]]
                  [--dataset_name DATASET_NAME]
                  [--dataset_config_name DATASET_CONFIG_NAME]
                  [--train_file TRAIN_FILE]
                  [--validation_file VALIDATION_FILE]
                  [--block_size BLOCK_SIZE]
                  [--overwrite_cache [OVERWRITE_CACHE]]
                  [--validation_split_percentage VALIDATION_SPLIT_PERCENTAGE]
                  [--preprocessing_num_work

## 5. Generando texto

La sigueintes celdas generan el texto, la primera recupera el modelo entrenado desde nuestro google drive, y la segunda hace la ejecuión. Uno puede ejecutar la segunda opción tantas veces como vea uno necesario. La opción _max_length_ controla la cantidad de texto generado.

In [10]:
from transformers import pipeline
model = "/content/drive/MyDrive/models/gpt2-small-spanish-adaptado"
model_text = pipeline('text-generation',model=model, tokenizer=model,)

In [12]:
print(model_text("La muerte era una hermosa flor",max_length=1000)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


La muerte era una hermosa flor. --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  -- Tu aliento seductora --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  -- ¿Por qué no te has olvidado lo que has pasado --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --   --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  --  -- Va en constante movimiento --   --   --   --   --   --   --   -- 